## Sorting and tagging customers

## Sorting

In [1]:
def my_sort(xs,f=lambda x:x):
    n = len(xs)
    for i in range(n - 1):
        for j in range(0, n-i-1):
            if f(xs[j]) < f(xs[j+1]):
                xs[j],xs[j+1] =xs[j+1],xs[j]
                
    return xs             

In [2]:
xs = [1000,20000.35,3.6,40.6,50,6000,70]
my_sort(xs)

[20000.35, 6000, 1000, 70, 50, 40.6, 3.6]

In [3]:
def my_sort_dict(xs, f = lambda x:x[1]):
    xs = [(k,v) for k,v in xs.items()]
    return {k: v for (k,v) in my_sort(xs,f)}
    

In [4]:
test_dict = {"customer1":9000, "customer2":5000}
test_dict


{'customer1': 9000, 'customer2': 5000}

In [5]:
test_dict.items()

dict_items([('customer1', 9000), ('customer2', 5000)])

In [6]:
my_sort_dict(test_dict)

{'customer1': 9000, 'customer2': 5000}

## Applying to our dataset

In [7]:
import pandas as pd
import sqlite3

In [8]:
conn = sqlite3.connect("churn.db")

In [9]:
LTV= pd.read_sql("select CustomerID,TotalCharges from churn_all",conn)

In [10]:
LTV.TotalCharges

0        108.15
1        151.65
2         820.5
3       3046.05
4        5036.3
         ...   
7038     1419.4
7039     1990.5
7040     7362.9
7041     346.45
7042     6844.5
Name: TotalCharges, Length: 7043, dtype: object

In [11]:
LTV["TotalCharges"] = pd.to_numeric(LTV["TotalCharges"],errors="coerce")

In [12]:
LTV

,CustomerID,TotalCharges
0,3668-QPYBK,108.15
1,9237-HQITU,151.65
2,9305-CDSKC,820.50
3,7892-POOKP,3046.05
4,0280-XJGEX,5036.30
...,...,...
7038,2569-WGERO,1419.40
7039,6840-RESVB,1990.50
7040,2234-XADUH,7362.90
7041,4801-JZAZL,346.45


In [13]:
LTV = LTV.set_index("CustomerID").T.to_dict("records")[0]

In [14]:
LTV

{'3668-QPYBK': 108.15,
 '9237-HQITU': 151.65,
 '9305-CDSKC': 820.5,
 '7892-POOKP': 3046.05,
 '0280-XJGEX': 5036.3,
 '4190-MFLUW': 528.35,
 '8779-QRDMV': 39.65,
 '1066-JKSGK': 20.15,
 '6467-CHFZW': 4749.15,
 '8665-UTDHZ': 30.2,
 '8773-HHUOZ': 1093.1,
 '6047-YHPVI': 316.9,
 '5380-WJKOV': 3549.25,
 '8168-UQWWF': 1105.4,
 '7760-OYPDY': 144.15,
 '9420-LOJKX': 1426.4,
 '7495-OOKFY': 633.3,
 '1658-BYGOY': 1752.55,
 '5698-BQJOH': 857.25,
 '5919-TMRGD': 79.35,
 '9191-MYQKX': 496.9,
 '8637-XJIVR': 927.35,
 '0278-YXOOG': 113.85,
 '4598-XLKNJ': 2514.5,
 '3192-NQECA': 7611.85,
 '0486-HECZI': 5238.9,
 '4846-WHAFZ': 2868.15,
 '5299-RULOA': 1064.65,
 '0404-SWRVG': 229.55,
 '4412-YLTKF': 2135.5,
 '6207-WIOLX': 1502.65,
 '3091-FYHKI': 35.45,
 '2372-HWUHI': 81.25,
 '0390-DCFDQ': 70.45,
 '4080-OGPJL': 563.65,
 '2135-RXIHG': 45.65,
 '3874-EQOEP': 655.5,
 '0867-MKZVY': 1592.35,
 '3376-BMGFE': 273.0,
 '3445-HXXGF': 2651.2,
 '1875-QIVME': 242.8,
 '0691-JVSYA': 5000.2,
 '2656-FMOKZ': 1145.7,
 '2070-FNEXE': 503

In [15]:
my_sort_dict(LTV)

{'2889-FPWRM': 8684.8,
 '4376-KFVRS': 8468.2,
 '1480-BKXGA': 8404.9,
 '9680-NIAUV': 8129.3,
 '0201-OAMXR': 8127.6,
 '3520-FJGCV': 8126.65,
 '3886-CERTZ': 8109.8,
 '7255-SSFBC': 8041.65,
 '5168-MQQCA': 8003.8,
 '1444-VVSGW': 7968.85,
 '1679-JRFBR': 7930.55,
 '3655-SNQYZ': 7895.15,
 '6614-FHDBO': 7878.3,
 '5271-YNWVR': 7856.0,
 '5954-BDFSG': 7853.7,
 '8199-ZLLSA': 7804.15,
 '9053-JZFKV': 7752.3,
 '1555-DJEQW': 7723.9,
 '3259-FDWOY': 7723.7,
 '7317-GGVPB': 7690.9,
 '0917-EZOLA': 7689.95,
 '1984-FCOWB': 7674.55,
 '3192-NQECA': 7611.85,
 '1090-ESELR': 7611.55,
 '5287-QWLKY': 7548.1,
 '0748-RDGGM': 7534.65,
 '2834-JRTUA': 7532.15,
 '9835-ZIITK': 7491.75,
 '2659-VXMWZ': 7482.1,
 '9848-JQJTX': 7459.05,
 '5502-RLUYV': 7446.9,
 '7632-MNYOY': 7432.05,
 '3363-DTIVD': 7405.5,
 '9959-WOFKT': 7382.25,
 '8809-RIHDD': 7372.65,
 '5748-RNCJT': 7348.8,
 '5440-FLBQG': 7318.2,
 '6728-DKUCO': 7303.05,
 '9090-SGQXL': 7299.65,
 '6305-YLBMM': 7262.0,
 '6234-RAAPL': 7251.7,
 '4853-RULSV': 7250.15,
 '8634-CILSZ':

In [65]:
df = pd.read_sql("select * from churn_all",conn)

## Tagging customers

In [16]:
def create_seg(df,LTV_col,target_col,LTV_value,upper_seg = "High",lower_seg ="Low"):
    df[target_col]= df[LTV_col].map(lambda x:upper_seg if x > LTV_value else lower_seg)
    return df

In [17]:
df = pd.read_sql("select * from churn_all",conn)

In [18]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"], errors="coerce")

In [19]:
LTV_value = df.TotalCharges.quantile(0.80)

In [20]:
churn_tagged = create_seg(df,"TotalCharges","LTV_segment",LTV_value)

In [21]:
churn_tagged.LTV_segment.value_counts()

Low     5636
High    1407
Name: LTV_segment, dtype: int64

## Creating the LTV_analysis class

In [71]:
class LTV_analysis:
    
    def __init__(self,df):
         self.df =df
        
    
    def my_sort(self,xs,f=lambda x:x):
        n = len(xs)
        for i in range(n - 1):
            for j in range(0, n-i-1):
                if f(xs[j]) < f(xs[j+1]):
                    xs[j],xs[j+1] =xs[j+1],xs[j]

        return xs 
   
    def my_sort_dict(self,xs, f = lambda x:x[1]):
        xs = [(k,v) for k,v in xs.items()]
        return {k: v for (k,v) in my_sort(xs,f)}
    
    def create_seg(self,df,LTV_col,target_col,LTV_value,upper_seg = "High",lower_seg ="Low"):
        self.df[target_col]= self.df[LTV_col].map(lambda x:upper_seg if x > LTV_value else lower_seg)
        return df
    
   
    
    

In [77]:
l = LTV_analysis(df)

In [61]:
l.my_sort_dict(LTV)

{'2889-FPWRM': 8684.8,
 '4376-KFVRS': 8468.2,
 '1480-BKXGA': 8404.9,
 '9680-NIAUV': 8129.3,
 '0201-OAMXR': 8127.6,
 '3520-FJGCV': 8126.65,
 '3886-CERTZ': 8109.8,
 '7255-SSFBC': 8041.65,
 '5168-MQQCA': 8003.8,
 '1444-VVSGW': 7968.85,
 '1679-JRFBR': 7930.55,
 '3655-SNQYZ': 7895.15,
 '6614-FHDBO': 7878.3,
 '5271-YNWVR': 7856.0,
 '5954-BDFSG': 7853.7,
 '8199-ZLLSA': 7804.15,
 '9053-JZFKV': 7752.3,
 '1555-DJEQW': 7723.9,
 '3259-FDWOY': 7723.7,
 '7317-GGVPB': 7690.9,
 '0917-EZOLA': 7689.95,
 '1984-FCOWB': 7674.55,
 '3192-NQECA': 7611.85,
 '1090-ESELR': 7611.55,
 '5287-QWLKY': 7548.1,
 '0748-RDGGM': 7534.65,
 '2834-JRTUA': 7532.15,
 '9835-ZIITK': 7491.75,
 '2659-VXMWZ': 7482.1,
 '9848-JQJTX': 7459.05,
 '5502-RLUYV': 7446.9,
 '7632-MNYOY': 7432.05,
 '3363-DTIVD': 7405.5,
 '9959-WOFKT': 7382.25,
 '8809-RIHDD': 7372.65,
 '5748-RNCJT': 7348.8,
 '5440-FLBQG': 7318.2,
 '6728-DKUCO': 7303.05,
 '9090-SGQXL': 7299.65,
 '6305-YLBMM': 7262.0,
 '6234-RAAPL': 7251.7,
 '4853-RULSV': 7250.15,
 '8634-CILSZ':

In [73]:
df["TotalCharges"] = pd.to_numeric(df["TotalCharges"],errors="coerce")

In [74]:
LTV_value = df.TotalCharges.quantile(0.80)

In [78]:
l.create_seg(df,"TotalCharges","LTV_segment",LTV_value)

,CustomerID,Gender,SeniorCitizen,Partner,Dependents,State,Latitude,Longitude,ZipCode,PhoneService,...,StreamingTV,StreamingMovies,Tenure,Contract,PaymentMethod,PaperlessBilling,MonthlyCharges,TotalCharges,Churn,LTV_segment
0,3668-QPYBK,Male,No,No,No,California,33.964131,-118.272783,90003,Yes,...,No,No,2,Month-to-month,Mailed check,Yes,53.85,108.15,Yes,Low
1,9237-HQITU,Female,No,No,Yes,California,34.059281,-118.307420,90005,Yes,...,No,No,2,Month-to-month,Electronic check,Yes,70.70,151.65,Yes,Low
2,9305-CDSKC,Female,No,No,Yes,California,34.048013,-118.293953,90006,Yes,...,Yes,Yes,8,Month-to-month,Electronic check,Yes,99.65,820.50,Yes,Low
3,7892-POOKP,Female,No,Yes,Yes,California,34.062125,-118.315709,90010,Yes,...,Yes,Yes,28,Month-to-month,Electronic check,Yes,104.80,3046.05,Yes,Low
4,0280-XJGEX,Male,No,No,Yes,California,34.039224,-118.266293,90015,Yes,...,Yes,Yes,49,Month-to-month,Bank transfer (automatic),Yes,103.70,5036.30,Yes,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,2569-WGERO,Female,No,No,No,California,34.341737,-116.539416,92285,Yes,...,No internet service,No internet service,72,Two year,Bank transfer (automatic),Yes,21.15,1419.40,No,Low
7039,6840-RESVB,Male,No,Yes,Yes,California,34.667815,-117.536183,92301,Yes,...,Yes,Yes,24,One year,Mailed check,Yes,84.80,1990.50,No,Low
7040,2234-XADUH,Female,No,Yes,Yes,California,34.559882,-115.637164,92304,Yes,...,Yes,Yes,72,One year,Credit card (automatic),Yes,103.20,7362.90,No,High
7041,4801-JZAZL,Female,No,Yes,Yes,California,34.167800,-116.864330,92305,No,...,No,No,11,Month-to-month,Electronic check,Yes,29.60,346.45,No,Low
